In [1]:
using QuantumToolbox
using Dates

function build_system(N, wc, J, cutoff)
    # QuantumToolbox mimics QuTiP:
    # cutoff=3 creates a basis of size 3 (levels 0, 1, 2)
    # So we DO NOT subtract 1 here (unlike QuantumOptics.jl)
    
    a = destroy(cutoff)
    at = create(cutoff)
    n = number(cutoff)
    id = qeye(cutoff)

    # Initialize Operator List
    # We build a list of terms and sum them up at the end
    # This is often more efficient than H += term in a loop for this library
    ops = []

    # 1. Self Energy
    for i in 1:N
        op_list = [id for _ in 1:N]
        op_list[i] = n
        push!(ops, wc * tensor(op_list...))
    end

    # 2. Hopping
    for i in 1:N-1
        # Term: a_i^dag * a_{i+1}
        op1 = [id for _ in 1:N]
        op1[i] = at
        op1[i+1] = a
        
        # Term: a_i * a_{i+1}^dag
        op2 = [id for _ in 1:N]
        op2[i] = a
        op2[i+1] = at
        
        # Add hopping interaction
        push!(ops, -J * (tensor(op1...) + tensor(op2...)))
    end

    # Sum all terms to get Hamiltonian
    return sum(ops)
end

function main()
    # Parameters
    wc = 1.0
    J = 0.1
    cutoff = 3
    tlist = [0:0.5:50;] # Time list 0 to 50
    
    cavities = 2:14
    times_toolbox = Float64[]

    println("Benchmarking QuantumToolbox.jl (N=2 to 14)...")
    println("---------------------------------------------")

    for N in cavities
        GC.gc() # Force garbage collection
        t_start = now()

        # 1. Build Hamiltonian
        H = build_system(N, wc, J, cutoff)

        # 2. Initial State |1, 0, 0...>
        psi_list = [basis(cutoff, 0) for _ in 1:N]
        psi_list[1] = basis(cutoff, 1)
        psi0 = tensor(psi_list...)

        # 3. Run Dynamics (sesolve = Schrodinger Equation Solve)
        # Note: QuantumToolbox compiles the first time it runs, 
        # so N=2 might seem slightly slow due to compilation overhead.
        sesolve(H, psi0, tlist)

        dt = (now() - t_start).value / 1000.0
        push!(times_toolbox, dt)
        
        println("N=$N: $dt s")
    end

    println("\n--- COPY THIS LIST ---")
    println(times_toolbox)
    println("----------------------")
end

main()

Benchmarking QuantumToolbox.jl (N=2 to 14)...
---------------------------------------------


LoadError: UndefVarError: `number` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

  Julia 1.12 has introduced more strict world age semantics for global bindings.
  !!! This code may malfunction under Revise.
  !!! This code will error in future versions of Julia.
Hint: Add an appropriate `invokelatest` around the access to this binding.
To make this warning an error, and hence obtain a stack trace, use `julia --depwarn=error`.
  Julia 1.12 has introduced more strict world age semantics for global bindings.
  !!! This code may malfunction under Revise.
  !!! This code will error in future versions of Julia.
Hint: Add an appropriate `invokelatest` around the access to this binding.
To make this warning an error, and hence obtain a stack trace, use `julia --depwarn=error`.


In [2]:
using QuantumToolbox
using Dates

function build_system(N, wc, J, cutoff)
    a = destroy(cutoff)
    at = create(cutoff)
    n = num(cutoff)
    id = qeye(cutoff)

    ops = []

    # 1. Self Energy
    for i in 1:N
        # FIX: Use Any[...] to allow mixing Diagonal (id) and Sparse (n) matrices
        op_list = Any[id for _ in 1:N]
        op_list[i] = n
        push!(ops, wc * tensor(op_list...))
    end

    # 2. Hopping
    for i in 1:N-1
        # Term 1: a_i^dag * a_{i+1}
        # FIX: Use Any[...] here too
        op1 = Any[id for _ in 1:N]
        op1[i] = at
        op1[i+1] = a
        
        # Term 2: a_i * a_{i+1}^dag
        # FIX: Use Any[...] here too
        op2 = Any[id for _ in 1:N]
        op2[i] = a
        op2[i+1] = at
        
        push!(ops, -J * (tensor(op1...) + tensor(op2...)))
    end

    return sum(ops)
end

function main()
    # Parameters
    wc = 1.0
    J = 0.1
    cutoff = 3
    tlist = [0:0.5:50;] 
    
    cavities = 2:14
    times_toolbox = Float64[]

    println("Benchmarking QuantumToolbox.jl (N=2 to 14)...")
    println("---------------------------------------------")

    for N in cavities
        GC.gc() 
        t_start = now()

        H = build_system(N, wc, J, cutoff)

        psi_list = [basis(cutoff, 0) for _ in 1:N]
        psi_list[1] = basis(cutoff, 1)
        psi0 = tensor(psi_list...)

        sesolve(H, psi0, tlist)

        dt = (now() - t_start).value / 1000.0
        push!(times_toolbox, dt)
        
        println("N=$N: $dt s")
    end

    println("\n--- COPY THIS LIST ---")
    println(times_toolbox)
    println("----------------------")
end

main()

Benchmarking QuantumToolbox.jl (N=2 to 14)...
---------------------------------------------


[sesolve] 100%|███████████████████████████| Time: 0:00:00 (49.51 μs/it)


N=2: 0.033 s


[sesolve] 100%|███████████████████████████| Time: 0:00:00 ( 0.00 ns/it)


N=3: 0.002 s


...

N=12: 15.0 s


[sesolve] 100%|███████████████████████████| Time: 0:00:38 ( 0.38  s/it)
[sesolve]   2%|█                          |  ETA: 0:09:23 ( 5.69  s/it)

N=13: 43.268 s


[sesolve] 100%|███████████████████████████| Time: 0:02:05 ( 1.24  s/it)

N=14: 142.099 s

--- COPY THIS LIST ---
[0.033, 0.002, 0.1, 0.012, 0.029, 0.067, 0.173, 0.424, 1.455, 5.115, 15.0, 43.268, 142.099]
----------------------
